In [11]:
# importing necessary packages
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn import linear_model
import matplotlib.pyplot as plt

# notebook settings
pd.set_option('display.max_columns', None)

In [12]:
def coef_graph():
    # plotting columns and their coef impact on predictions
    coefs = pd.DataFrame(
    regr.coef_,
    #  columns=['Coefficients'], index=X_train.columns
    columns=['Coefficients'], index=X.columns
    )

    # sort coefficients highest to lowest
    coefs.sort_values(by=['Coefficients'], inplace=True)

    # function to add value labels
    def addlabels(x,y):
        for i in range(len(x)):
            plt.text(i,y[i],y[i])

    # creating the graph
    coefs.plot(kind='barh', figsize=(9, 7))
    plt.title('Linear Regression Model')
    plt.axvline(x=0, color='.5')
    plt.subplots_adjust(left=.3)

# Data Preprocessing

In [13]:
ml_df = pd.read_csv('./data/machine-learning.csv', index_col=0)

In [18]:
# Splitting dataset for predictions
y = ml_df['C-Deaths']
X = ml_df.drop(['C-Deaths', 'Deaths'], axis=1)

In [20]:
# Scaling X learning data
scaler = preprocessing.MinMaxScaler()
names = X.columns
d = scaler.fit_transform(X)
X = pd.DataFrame(d, columns=names)

# Multifactor Regression Approach

In [25]:
# Splitting X and y into training datasets
kf = KFold(n_splits=10, shuffle=True)
# kf = KFold(n_splits=5,random_state=42,shuffle=True)
for train_index,val_index in kf.split(X):
    X_train,X_test = X.iloc[train_index],X.iloc[val_index],
    y_train,y_test = y.iloc[train_index],y.iloc[val_index]

    regr = linear_model.LinearRegression()
    regr.fit(X_train, y_train)

    # testing model for general accuracy
    print(f'\nModel score on testing data: {regr.score(X_test, y_test)}')
    print(f'Model score on training data: {regr.score(X_train, y_train)}')

    # coef_graph()


Model score on testing data: 0.38144288831807405
Model score on training data: 0.6243210454483263

Model score on testing data: 0.40131687160494156
Model score on training data: 0.6209081231045419

Model score on testing data: 0.3200122182941051
Model score on training data: 0.6206508827113961

Model score on testing data: 0.20705092799609326
Model score on training data: 0.6272978861855582

Model score on testing data: 0.6878308836175273
Model score on training data: 0.5780414361689814

Model score on testing data: 0.8130636687255717
Model score on training data: 0.5634330861416024

Model score on testing data: 0.508966201721859
Model score on training data: 0.6140222007405991

Model score on testing data: 0.31022258082370213
Model score on training data: 0.6231426489634019

Model score on testing data: 0.4141140891925482
Model score on training data: 0.6216749834402713

Model score on testing data: 0.469280725956587
Model score on training data: 0.6173177779509321
